In [1]:
%matplotlib inline
import pandas as pd

In [2]:
compras_df=pd.read_csv('/content/drive/MyDrive/PROYETO CLASE/compras_df_enriched.csv', encoding='windows-1252')

#PROBAR UN MODELO SUPERVISADO

In [6]:
data = compras_df.copy()
for i, column in enumerate(list([str(d) for d in data.dtypes])):
    if column == "object":
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].mode())
        data[data.columns[i]] = data[data.columns[i]].astype("category").cat.codes
    else:
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].median())

In [7]:
data.head(5)

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,ANO,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO,HAS_LINKEDIN_SITE
0,0,75,81,0,0,165,1,299000163.0,14,10997.79,2,16.0,12.0,2020.0,8.0,881659.731797,1
1,1,-1,-1,1,-1,-1,-1,299000163.0,-1,10997.79,-1,16.0,12.0,2020.0,8.0,881659.731797,-1
2,2,-1,-1,2,-1,-1,-1,299000163.0,-1,10997.79,-1,16.0,12.0,2020.0,8.0,881659.731797,-1
3,3,-1,-1,3,-1,-1,-1,299000163.0,-1,10997.79,-1,16.0,12.0,2020.0,8.0,881659.731797,-1
4,4,-1,-1,4,-1,-1,-1,299000163.0,-1,10997.79,-1,16.0,12.0,2020.0,8.0,881659.731797,-1


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 

In [10]:
y = data.pop('IMPORTE')
X = data.copy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
clf = RandomForestRegressor(n_estimators=1000, max_depth = 4)

In [13]:
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, n_estimators=1000)

In [14]:
from sklearn.metrics import r2_score

In [15]:
r2_score(y_test, clf.predict(X_test))

0.3511046287046281

## Probar un modelo no supervisado o de clustering para texto

In [16]:
import gensim
from nltk.corpus import stopwords
import numpy as np
np.random.seed(2021)

In [18]:
def preprocess(text):
    result = []
    for token in text.split():
        if token not in stopwords.words('spanish') and len(token)>4:
            result.append(token)
    return result

In [29]:
# agrupamos los valores
compras_df_c_grupo = compras_df.groupby('BENEFICIARIO').agg({'CONTRATO': lambda x: x.nunique() , 'IMPORTE': 'sum',
                                                               'FACTURA': lambda x: x.nunique()})
compras_df_cluster = compras_df_c_grupo.reset_index()
compras_df_cluster

,BENEFICIARIO,CONTRATO,IMPORTE,FACTURA
0,ARRIAGA PEREZ ROCIO JAZMIN,1,44537.96,2
1,ASOCIACION DE COLONOS CONTRY LA SILLA A.C.,1,70000.00,2
2,AYALA SALAZAR GABRIEL,1,44602.65,1
3,BANCO MERCANTIL DEL NORTE S.A. IBM GFB,1,25080.36,1
4,BRANDI ELGUEZABAL MARCELO,1,90591.95,2
...,...,...,...,...
89,VALLEJO RAMOS JUAN MANUEL,1,36104.10,2
90,VARGAS ANTILLON MARCOS,1,3053.46,1
91,VARGAS GARCIA ALHINNA BERENICE,1,41704.40,1
92,VILLARREAL CARDENAS JUAN IGNACIO,1,3733.13,1


In [30]:
compras_df_c_grupo_f = compras_df_c_grupo.reset_index().drop(['BENEFICIARIO'], axis = 1)

normalizado_compras_df = (compras_df_c_grupo_f- compras_df_c_grupo_f.min())/(compras_df_c_grupo_f.max()-compras_df_c_grupo_f.min())
normalizado_compras_df

,CONTRATO,IMPORTE,FACTURA
0,0.0,0.003560,0.020408
1,0.0,0.005619,0.020408
2,0.0,0.003565,0.000000
3,0.0,0.001987,0.000000
4,0.0,0.007284,0.020408
...,...,...,...
89,0.0,0.002878,0.020408
90,0.0,0.000205,0.000000
91,0.0,0.003331,0.000000
92,0.0,0.000260,0.000000


 ## Formulando conclusiones
.Plantear la tendencia central

.Hay historias en los outeliers

.Esfuerzate en contestar la pregunta original

.Si tu enfoque cambio, documentalo.